# Credential Access Threat Hunting - LSASS Credential Dumping
### Author: Bintang N M
The Windows Task Manager may be used to dump the memory space of lsass.exe to disk for processing with a credential access tool such as Mimikatz. This is performed by launching Task Manager as a privileged user, selecting lsass.exe, and clicking “Create dump file”. This saves a dump file to disk with a deterministic name that includes the name of the process being dumped.

This requires filesystem data to determine whether files have been created.

## Use Case
### Objective
Identify LSASS memory dumping. LSASS memory may contain many credentials including logged on users clear-text passwords, Kerberos tickets, etc. 

### Logs Requirements
Microsoft Windows Sysmon

### References
MITRE references: https://car.mitre.org/analytics/CAR-2019-08-001/ <br>
sysmon file as json: https://raw.githubusercontent.com/OTRF/mordor/master/datasets/small/windows/credential_access/host/rdp_interactive_taskmanager_lsass_dump.zip

In [1]:
import evtx
import xml.etree.ElementTree as ET
import json
import re

In [4]:
evtx_file = 'logs/Credential Access/rdp_interactive_taskmanager_lsass_dump_2020-09-22043748.json'

In [5]:
with open(evtx_file, 'r') as f:
    t = f.read().split('\n')
    
x = json.loads(json.dumps(list(t)))

## Look for .dmp file creation from task manager
File creation event id is 11<br>
Use regex to search .dmp file for element EventData.TargetFilename

In [13]:
temp_events = []
error_datas = []

for xs in x:
    try:
        if (json.loads(xs)['EventID'] == 11): #filter by event id
            if re.search("dmp", json.loads(xs)['TargetFilename'], re.IGNORECASE): #filter dmp on TargetFilename using regex
                temp_events += [xs]
    except:
        error_datas += [xs]
print(len(temp_events))

1


In [14]:
json.loads(temp_events[0])

{'Keywords': -9223372036854775808,
 'SeverityValue': 2,
 'EventReceivedTime': '2020-09-22 04:38:29',
 'EventID': 11,
 'ProviderGuid': '{5770385F-C22A-43E0-BF4C-06F5698FFBD9}',
 'ExecutionProcessID': 3172,
 'Channel': 'Microsoft-Windows-Sysmon/Operational',
 'host': 'wec.internal.cloudapp.net',
 'AccountType': 'User',
 'UserID': 'S-1-5-18',
 'ProcessGuid': '{b34bc01c-b7f3-5f69-df02-000000000900}',
 'ThreadID': 4048,
 'EventType': 'INFO',
 'Opcode': 'Info',
 'EventTime': '2020-09-22 04:38:27',
 '@timestamp': '2020-09-22T08:38:29.039Z',
 'TargetFilename': 'C:\\Users\\pgustavo\\AppData\\Local\\Temp\\lsass.DMP',
 'SourceModuleType': 'im_msvistalog',
 'port': 64545,
 'AccountName': 'SYSTEM',
 'RecordNumber': 3407010,
 'Task': 11,
 'Domain': 'NT AUTHORITY',
 '@version': '1',
 'OpcodeValue': 0,
 'CreationUtcTime': '2020-09-22 08:38:27.371',
 'SourceModuleName': 'eventlog',
 'Severity': 'INFO',
 'SourceName': 'Microsoft-Windows-Sysmon',
 'Version': 2,
 'Image': 'C:\\windows\\system32\\taskmgr.e

## Look for process access event on lsass.exe
Process access event id is 10<br>
Use regex to search lsass.exe file for element EventData.TargetFilename<br>
Use regex to look into EventData.SourceImage for {"taskmgr.exe", "procexp64.exe", "procexp.exe", "lsm.exe", "csrss.exe", "wininit.exe", "wmiprvse.exe"}

In [17]:
temp_event = []
error_data = []

for xs in x:
    try:
        #if clause for filtering event ID and using regex for filtering "lsass.exe" from TargetImage
        if (json.loads(xs)['EventID'] == 10) and (re.search("lsass.exe\Z", json.loads(xs)['TargetImage'])):
            #if clause for filtering event using regex
            if re.search("taskmgr.exe\Z|procexp64.exe\Z|procexp.exe\Z|lsm.exe\Z|csrss.exe\Z|wininit.exe\Z|wmiprvse.exe\Z", json.loads(xs)['SourceImage'], re.IGNORECASE):
                temp_event += [xs]
    except:
        error_data += [xs]
print(len(temp_event))

7


In [18]:
json.loads(temp_event[0])

{'Keywords': -9223372036854775808,
 'SeverityValue': 2,
 'SourceImage': 'C:\\windows\\system32\\taskmgr.exe',
 'EventID': 10,
 'ProviderGuid': '{5770385F-C22A-43E0-BF4C-06F5698FFBD9}',
 'ExecutionProcessID': 3172,
 'Channel': 'Microsoft-Windows-Sysmon/Operational',
 'host': 'wec.internal.cloudapp.net',
 'AccountType': 'User',
 'UserID': 'S-1-5-18',
 'SourceProcessGUID': '{b34bc01c-b7f3-5f69-df02-000000000900}',
 'ThreadID': 4048,
 'TargetImage': 'C:\\windows\\system32\\lsass.exe',
 'GrantedAccess': '0x1400',
 'EventType': 'INFO',
 'Opcode': 'Info',
 'EventTime': '2020-09-22 04:38:12',
 'EventReceivedTime': '2020-09-22 04:38:16',
 '@timestamp': '2020-09-22T08:38:16.590Z',
 'SourceModuleType': 'im_msvistalog',
 'port': 64545,
 'AccountName': 'SYSTEM',
 'RecordNumber': 3406235,
 'SourceProcessId': '624',
 'SourceThreadId': '5452',
 'Task': 10,
 'Domain': 'NT AUTHORITY',
 '@version': '1',
 'OpcodeValue': 0,
 'SourceModuleName': 'eventlog',
 'TargetProcessGUID': '{b34bc01c-226a-5f69-0c00-00

## EVTX
The exampe case used json as dataset, use these codes when using evtx file

In [ ]:
evtx_file = 'evtx_address' #replace evtx_address with evtx file address

#### Filtering events based on event ID

In [ ]:
parser = evtx.PyEvtxParser(evtx_file)
parse_json = list(parser.records_json())

temp_events = []

for pj in parse_json:
    if '"EventID": {event_id}' in pj['data']:#replace event_id with. Try using regex if you're unsure about the result.
        temp_events+=[pj]
        print(pj['data'])

#### Filtering other elements
To filter other elements, make sure you are loading the 'data' element after parsing the evtx to json. Then filter each case accodringly.<br>
Example for finding .dmp file creation:

In [ ]:
temp_events = []
error_datas = []

for pj in parse_json:
    try:
        if re.search("dmp", json.loads(pj['data'])['EventData']['TargetFilename'], re.IGNORECASE):
            temp_events += [pj]
    except:
        error_datas += [pj]
print(len(temp_events))